This model shows how to train BertSum model and evaluate the model on ThaiSum dataset. More details on README.md.

# Install requirements

In [1]:
%%capture
!git clone https://github.com/nakhunchumpolsathien/ThaiSum.git

In [2]:
!pip install -q pytorch_transformers
!pip install -q tensorboardX
!pip install -q pyrouge
!pyrouge_set_rouge_path "/content/ThaiSum/BertSum/ROUGE-1.5.5"
!apt update
!apt install -q libxml-parser-perl
%cd "/content/ThaiSum/BertSum/ROUGE-1.5.5/data"
!perl WordNet-2.0-Exceptions/buildExeptionDB.pl ./WordNet-2.0-Exceptions ./smart_common_words.txt ./WordNet-2.0.exc.db
%cd '/content/ThaiSum/BertSum/src'

2020-11-13 14:50:01,979 [MainThread  ] [INFO ]  Set ROUGE home directory to /content/ThaiSum/BertSum/ROUGE-1.5.5.
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Reading package 

# Preprocessing Data


## Step 5. Format to PyTorch Files
```
!python preprocess.py -mode format_to_bert -raw_path "/content/BertSum/simple_js/thaisum" -save_path "/content/BertSum/bert_data/thaisum"  -lower -n_cpus 1 -log_file ../logs/preprocess.log
```

In [3]:
# Step 5. Format to PyTorch Files
!python preprocess.py -mode format_to_bert -raw_path "/content/ThaiSum/BertSum/simple_js" -save_path "/content/ThaiSum/BertSum/bert_data"  -lower -n_cpus 1 -log_file "/content/ThaiSum/BertSum/logs/thaisum_preprocess.txt"

[('train', '/content/ThaiSum/BertSum/simple_js/thaisum.train.0.json', Namespace(dataset='', log_file='/content/ThaiSum/BertSum/logs/thaisum_preprocess.txt', lower=True, map_path='../../data/', max_src_nsents=100, max_src_ntokens_per_sent=200, max_tgt_ntokens=500, min_src_nsents=3, min_src_ntokens_per_sent=5, min_tgt_ntokens=5, mode='format_to_bert', n_cpus=1, pretrained_model='bert', raw_path='/content/ThaiSum/BertSum/simple_js', save_path='/content/ThaiSum/BertSum/bert_data', select_mode='greedy', shard_size=2000, use_bert_basic_tokenizer=False), '/content/ThaiSum/BertSum/bert_data/thaisum.train.0.bert.pt')]
[2020-11-13 14:51:14,210 INFO] loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /root/.cache/torch/pytorch_transformers/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729
[2020-11-13 14:51:14,362 INFO] Processing /content/Th

# Train Model
You can train BertSum model on Google Colab with one GPU, especially BertSumExt,  just try to adjust the parameters.

## Train BertSumExt

```
python train.py -task ext -mode train -bert_data_path BERT_DATA_PATH -ext_dropout 0.1 -model_path MODEL_PATH -lr 2e-3 -visible_gpus 0,1,2 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -train_steps 50000 -accum_count 2 -log_file ../logs/ext_bert_cnndm -use_interval true -warmup_steps 10000 -max_pos 512
```



In [4]:
!python train.py -task ext -mode train -bert_data_path "/content/ThaiSum/BertSum/bert_data/thaisum" -ext_dropout 0.1 -model_path "/content/ARedSum/model_checkpoint/BertSumExt" -lr 2e-3 -visible_gpus 0 -report_every 50 -save_checkpoint_steps 1000 -batch_size 3000 -train_steps 50000 -accum_count 2 -log_file ../logs/train_bertsumext_thaisum -use_interval true -warmup_steps 10000 -max_pos 512

[2020-11-13 14:51:57,987 INFO] Device ID 0
[2020-11-13 14:51:57,987 INFO] Device cuda
[2020-11-13 14:51:58,119 INFO] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-config.json from cache at ../temp/45629519f3117b89d89fd9c740073d8e4c1f0a70f9842476185100a8afe715d1.65df3cef028a0c91a7b059e4c404a975ebe6843c71267b67019c0e9cfa8a88f0
[2020-11-13 14:51:58,120 INFO] Model config {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attent

## Train BertSumAbs

```
python train.py  -task abs -mode train -bert_data_path BERT_DATA_PATH -dec_dropout 0.2  -model_path MODEL_PATH -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 2000 -batch_size 140 -train_steps 200000 -report_every 50 -accum_count 5 -use_bert_emb true -use_interval true -warmup_steps_bert 20000 -warmup_steps_dec 10000 -max_pos 512 -visible_gpus 0,1,2,3  -log_file ../logs/abs_bert_cnndm
```



## Train BertSumExtAbs
```
python train.py  -task abs -mode train -bert_data_path BERT_DATA_PATH -dec_dropout 0.2  -model_path MODEL_PATH -sep_optim true -lr_bert 0.002 -lr_dec 0.2 -save_checkpoint_steps 2000 -batch_size 140 -train_steps 200000 -report_every 50 -accum_count 5 -use_bert_emb true -use_interval true -warmup_steps_bert 20000 -warmup_steps_dec 10000 -max_pos 512 -visible_gpus 0,1,2,3 -log_file ../logs/abs_bert_cnndm  -load_from_extractive EXT_CKPT   
```


# Evaluate the model 
You can download trained BertSum models from https://bit.ly/bertsum.

## Evaluate with ROUGE
[Original ROUGE Script](https://github.com/bheinzerling/pyrouge)

In [5]:
!python train.py -task ext -mode test -batch_size 3000 -test_batch_size 500 -bert_data_path "/content/ThaiSum/BertSum/bert_data/thaisum" -log_file ../logs/eval_bertsumext -model_path "/content/ThaiSum/BertSum/model_checkpoint/" -sep_optim true -use_interval true -visible_gpus 0 -max_pos 512 -max_length 200 -alpha 0.95 -min_length 50 -result_path ../results/bertsumext -test_from "/content/ThaiSum/BertSum/model_checkpoint/BertsumExt_thaisum.pt"

[2020-11-13 14:54:03,428 INFO] Loading checkpoint from /content/ThaiSum/BertSum/model_checkpoint/BertsumExt_thaisum.pt
Namespace(accum_count=1, alpha=0.95, batch_size=3000, beam_size=5, bert_data_path='/content/ThaiSum/BertSum/bert_data/thaisum', beta1=0.9, beta2=0.999, block_trigram=True, dec_dropout=0.2, dec_ff_size=2048, dec_heads=8, dec_hidden_size=768, dec_layers=6, enc_dropout=0.2, enc_ff_size=512, enc_hidden_size=512, enc_layers=6, encoder='bert', ext_dropout=0.2, ext_ff_size=2048, ext_heads=8, ext_hidden_size=768, ext_layers=2, finetune_bert=True, generator_shard_size=32, gpu_ranks=[0], label_smoothing=0.1, large=False, load_from_extractive='', log_file='../logs/eval_bertsumext', lr=1, lr_bert=0.002, lr_dec=0.002, max_grad_norm=0, max_length=200, max_pos=512, max_tgt_len=140, min_length=50, mode='test', model_path='/content/ThaiSum/BertSum/model_checkpoint/', optim='adam', param_init=0, param_init_glorot=True, recall_eval=False, report_every=1, report_rouge=True, result_path='.

## Evaluate with BertScore 

In [6]:
!pip install -q bert_score
import bert_score
from bert_score import score
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

     |████████████████████████████████| 61kB 3.3MB/s 
     |████████████████████████████████| 1.3MB 6.1MB/s 
     |████████████████████████████████| 1.1MB 11.0MB/s 
     |████████████████████████████████| 2.9MB 21.3MB/s 


In [7]:
with open("/content/ThaiSum/BertSum/results/bertsumext_step0.candidate") as f: # Output Summary 
    cands = [line.strip() for line in f]

with open("/content/ThaiSum/BertSum/results/bertsumext_step0.gold") as f:  # Reference Summary
    refs = [line.strip() for line in f]

In [8]:
P, R, F1 = score(cands, refs, lang='th', verbose=False)

print(f"System level F1 score: {F1.mean()*100:.3f}") ##  *100 to make it simpler to read similar to ROUGE.
print(f"System level P score: {P.mean()*100:.3f}")
print(f"System level R score: {R.mean()*100:.3f}")


System level F1 score: 92.794
System level P score: 94.092
System level R score: 91.614


/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,


## Show Output Summary

In [10]:
with open("/content/ThaiSum/BertSum/results/bertsumext_step0.candidate") as f:
    for line in f:
        print(line)

โควิด - 19 พ่น พิษ การ บิน ไทย ร่อน ไม่ ตรง รันเวย์ ทำ เที่ยวบิน และ รายได้ เดือน<q>มี .ค. - พ.ค. ลดลง 20 % แต่ ยัง ถูกหวย ขาย ที่นั่ง จอง ลอง เลก เพิ่ม รายได้ อย่าง น่าพิศวง

เปิดเผย ว่า จาก สถานการณ์ การ แพร่ระบาด ของ เชื้อไวรัส โควิด - 19 ที่ ยัง ไม่ ปรากฏ สัญญาณ ว่า จะ มีแนวโน้ม สิ้นสุด<q>ส่งผล ให้ ธุรกิจ ส่ง ออกดอก กล้วยไม้ รวมถึง กลุ่ม เกษตรกร ผู้ผลิต ได้รับ ผลกระทบ อย่าง หนัก

